In [ ]:
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 #accelerate==0.25.0
!pip install -q -U accelerate



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.1.1 requires transformers<5.0.0,>=4.38.0, but you have transformers 4.37.2 which is incompatible.


In [ ]:
# Install the required libraries
!pip install beautifulsoup4 langchain langchain-chroma langchain-community langchain-core langchain-openai langchain-groq
!pip install llama_index
!pip install langchain-text-splitters
!pip install llama-index-embeddings-huggingface
!pip install gradio
!pip install llama-index-llms-langchain
!pip install llama-index-embeddings-langchain
!pip install llama-index-llms-fireworks
!pip install llama-index-llms-groq
!pip install --upgrade llama-index


  Using cached transformers-4.45.1-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.45.1-py3-none-any.whl (9.9 MB)
Using cached tokenizers-0.20.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.9 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.core import VectorStoreIndex

from llama_index.core import SimpleDirectoryReader

from llama_index.core import Settings

from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.core.llms import ChatMessage



docs_both = SimpleDirectoryReader(input_files=["/content/data/PythonTB.pdf"]).load_data()

#sentence-transformers/all-mpnet-base-v2
#sentence-transformers/all-MiniLM-L6-V2
embed_model =  HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

import os

os.environ["GROQ_API_KEY"] = "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"


from llama_index.llms.groq import Groq

llm = Groq(model="llama3-8b-8192") #  llama-3.1-8b-instant
llm_70b = Groq(model="llama3-70b-8192")

Settings.llm = llm
Settings.embed_model = embed_model

messages = [
    ChatMessage(role="system", content="You are Athena, a friendly and supportive computer science teacher specializing in algorithms, data structures, and problem-solving, representing wisdom. Answer only computer science questions, avoid unrelated topics, and do not provide full answers to problem sets or coding solutions to maintain academic honesty. Instead, offer comments on how students can proceed or improve using bullet points, theoretical examples, and highlighting keywords in **bold**. If errors are spotted, mention the line numbers. Use simple language suitable for first-time learners, provide concise answers, avoid giving direct coding answers or corrected code, and allow follow-up questions without citing sources."),
    ChatMessage(role="user", content="Ask computer science related questions"),
]

index = VectorStoreIndex.from_documents(docs_both)
query_engine = index.as_query_engine(similarity_top_k=3)


<ipython-input-3-37a21443fda7>:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model =  HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will

In [ ]:
context_prompt = """
  The following is a friendly conversation between a user and an AI assistant named Athena.
  If the assistant does not know the answer to a question, it truthfully says it
  does not know. You don't give coding answers to the questions.

  Here are the relevant documents for the context:

  {context_str}

  Instruction: Based on the above documents, provide short-to-the-point explanations for the user question below but do not provide direct answers and coding solutions. Don't state where you extracted the answers from.
  Answer "don't know" if not present in the document.
  """
system_prompt = """You are a friendly and supportive computer science teacher that experts in teaching algorithms,
data structures, and problem-solving.
Your name is Athena and you represent wisdom. You don't know how to code: You only know how to explain.
Do not answer questions about topics other than computer science.
Do not provide full answers to problem sets as this would violate academic honesty.
Do not give coding solutions to the problems.
Use bullet points to point out the steps. Give theoretical examples of the techniques used but don't give the
solution.
Never include coding examples in the response.
Provide a summary in the beginning and make the keywords in bold.
If errors are spotted, give line numbers of the errors too.
Use language that could be easily understood by first-time programming learners. Never give direct answers to
coding questions.
Don't give any corrected versions of the wrong code provided by the user.
Keep your answers short to the point. Answer only the main idea of the question.
Allow the user to ask follow-up questions.
Don't state where you extracted your answer from.
Don't give coding examples, solutions or explanations.
You don't know how to code. You only know how to explain.
Don't apologize for not providing answers and coding solutions.
"""


memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    system_prompt=system_prompt,
    memory=memory,
    llm=llm,
    context_prompt= context_prompt,
    verbose=False,
)


In [ ]:
response = chat_engine.chat("How to reverse a string?")
print(response)



**Reversing a String**

Reversing a string means to change the order of its characters from the original sequence to the reverse sequence. For example, if you have the string "hello", reversing it would result in "olleh".

To reverse a string, you can use **slicing**. Slicing is a way to extract a subset of characters from a string. By using a slice that starts from the end of the string and moves backwards to the beginning, you can effectively reverse the string.

For example, if you have the string "hello", you can use the slice `[::-1]` to reverse it. This slice starts from the end of the string (`-1`) and moves backwards to the beginning, stepping backwards by 1 character each time (`-1` is the step size).

Here's a summary of the steps:

* Use slicing to extract a subset of characters from the string.
* Start the slice from the end of the string (`-1`).
* Move backwards to the beginning of the string.
* Step backwards by 1 character each time (`-1` is the step size).

By following

In [ ]:
response = chat_engine.chat("Explain step 2")
print(response)



**Step 2: Move backwards to the beginning of the string**

When we say "move backwards to the beginning of the string", we mean that we want to start from the last character of the string and move towards the first character. This is done by specifying the starting index of the slice as `-1`, which refers to the last character of the string.

Think of it like counting backwards from the end of the string. If the string is "hello", the last character is "o", so `-1` refers to "o". Then, we move backwards to the second-to-last character, which is "l", so `-2` refers to "l". We keep moving backwards until we reach the first character, which is "h", so `0` refers to "h".

By starting from the end of the string and moving backwards, we effectively reverse the order of the characters.


In [ ]:
#import gradio as gr


#def process_question(question):
   #response = chat_engine.chat(question)
   #return str(response)



#iface = gr.Interface(fn=process_question,
              #       inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),
              #       outputs=gr.Markdown(),
              #      title="Athena - AI Programming Assistant",
              #      description="Ask any question about your document, and get an answer along with the response time.")

# Launch the interface
#iface.launch()

In [ ]:
# import gradio as gr
# def update_prompt(topic):
#     return f"Generate 3 revision coding questions on {topic}"

# # Function to process the initial question
# def process_question(question):
#     response = chat_engine.chat(question)
#     return str(response)


# # List of topics
# beginner_topics = [
#     "Variables and Data Types",
#     "Control Structures",
#     "Functions",
#     "Basic Input/Output",
#     "Lists and Arrays",
#     "Dictionaries",
#     "String Manipulation",
#     "Basic Error Handling",
#     "Basic File Operations",
#     "Introduction to Object-Oriented Programming (OOP)",
#     "Basic Algorithm Concepts",
#     "Introduction to Libraries and Modules"
# ]

# advanced_topics = [
#     "Advanced Data Structures (Trees, Graphs)",
#     "Algorithms and Complexity (Big O Notation)",
#     "Recursion",
#     "Object-Oriented Design Principles",
#     "Design Patterns",
#     "Functional Programming Concepts",
#     "Concurrency and Multithreading",
#     "Memory Management and Optimization",
#     "Network Programming",
#     "Database Design and SQL",
#     "Web Development Frameworks",
#     "Machine Learning Basics"
# ]

# with gr.Blocks() as iface1:
#     with gr.Column():
#         gr.Markdown("# Athena - AI Programming Assistant")
#         gr.Markdown("## Ask any question related to computer science.")
#         input_box = gr.Textbox(lines=2, placeholder="Type your question here...", label="Question")
#         output_box = gr.Textbox(label="Answer")
#         generate_button = gr.Button("Generate")
#         generate_button.click(fn=process_question, inputs=input_box, outputs=output_box)



# with gr.Blocks() as iface2:
#     with gr.Column():
#         gr.Markdown("Test your skills here by asking Athena to give you revision questions. You can choose your topic from the buttons below or ask other topics.")



#         input_box2 = gr.Textbox(lines=2, placeholder="Generate revision questions on the topic here...", label="Revision Question")
#         output_box2 = gr.Textbox(label="Answer")
#         generate_button2 = gr.Button("Generate")




#         gr.Markdown("### Beginner Topics")
#         with gr.Row():
#             for topic in beginner_topics:
#                 button = gr.Button(topic, elem_id="topic-button")
#                 button.click(fn=update_prompt, inputs=gr.Textbox(value=topic, visible=False), outputs=input_box2)

#         gr.Markdown("### Advanced Topics")
#         with gr.Row():
#             for topic in advanced_topics:
#                 button = gr.Button(topic, elem_id="topic-button")
#                 button.click(fn=update_prompt, inputs=gr.Textbox(value=topic, visible=False), outputs=input_box2)
#         generate_button2.click(fn=process_question, inputs=input_box2, outputs=output_box2)



# # Create a Tabbed interface
# tabbed_iface = gr.TabbedInterface(
#     interface_list=[iface1, iface2],
#     tab_names=["Ask a Question", "Revise"]
# )

# # Launch the interface
# tabbed_iface.launch()

# # CSS for custom styling
# css = """
# #topic-button {
#     background-color: #4CAF50;
#     color: white;
#     border: none;
#     padding: 5px 10px;
#     text-align: center;
#     text-decoration: none;
#     display: inline-block;
#     margin: 2px;
#     cursor: pointer;
#     border-radius: 10px;
#     font-size: 8px;
# }

# #topic-button:hover {
#     background-color: #45a049;
# }
# """


# tabbed_iface.launch(inline=False, inbrowser=True, show_error=True,debug=True)

In [ ]:
import requests
from PIL import Image
import torch
from transformers import BitsAndBytesConfig
from transformers import pipeline
import gradio as gr


model_id = "llava-hf/llava-1.5-7b-hf"

# Set up 4-bit quantization for the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load the LLava model with quantization

pipe = pipeline(
    "image-to-text",
    model=model_id,
    model_kwargs={"quantization_config": quantization_config} #needed to not crash RAM
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Function to generate text from an uploaded image and question
def generate_response(image, user_prompt):
    prompt = f"USER: <image>\n{user_prompt}\nASSISTANT:"
    max_new_tokens = 200
    outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": max_new_tokens})
    generated_text = outputs[0]['generated_text']

    assistant_response = generated_text.split('ASSISTANT:')[1].strip()

    return assistant_response

# Function to update the prompt based on the selected topic
def update_prompt(topic):
    return f"Generate 3 revision coding questions on {topic}"

# Function to process the initial question in the "Ask a Question" tab
def process_question_with_image(image, question):
    if image:
        return generate_response(image, question)
    else:
        response = chat_engine.chat(question)
        return str(response)

# Function to process the initial question in the "Revise" tab
def process_question_no_image(question):
    response = chat_engine.chat(question)
    return str(response)


In [ ]:
# # List of topics
# beginner_topics = [
#     "Variables and Data Types",
#     "Control Structures",
#     "Functions",
#     "Basic Input/Output",
#     "Lists and Arrays",
#     "Dictionaries",
#     "String Manipulation",
#     "Basic Error Handling",
#     "Basic File Operations",
#     "Introduction to Object-Oriented Programming (OOP)",
#     "Basic Algorithm Concepts",
#     "Introduction to Libraries and Modules"
# ]

# advanced_topics = [
#     "Advanced Data Structures (Trees, Graphs)",
#     "Algorithms and Complexity (Big O Notation)",
#     "Recursion",
#     "Object-Oriented Design Principles",
#     "Design Patterns",
#     "Functional Programming Concepts",
#     "Concurrency and Multithreading",
#     "Memory Management and Optimization",
#     "Network Programming",
#     "Database Design and SQL",
#     "Web Development Frameworks",
#     "Machine Learning Basics"
# ]

# # Create the Gradio interfaces
# with gr.Blocks() as iface1:
#     with gr.Column():
#         gr.Markdown("# Athena - AI Programming Assistant")
#         gr.Markdown("## Ask any question related to computer science.")
#         image_input = gr.Image(type="pil", label="Upload an Image (Optional)")
#         input_box = gr.Textbox(lines=2, placeholder="Type your question here...", label="Question")
#         output_box = gr.Markdown()
#         generate_button = gr.Button("Generate")
#         generate_button.click(fn=process_question_with_image, inputs=[image_input, input_box], outputs=output_box)

# with gr.Blocks() as iface2:
#     with gr.Column():
#         gr.Markdown("Test your skills here by asking Athena to give you revision questions. You can choose your topic from the buttons below or ask about other topics.")
#         input_box2 = gr.Textbox(lines=2, placeholder="Generate revision questions on the topic here...", label="Revision Question")
#         output_box2 = gr.Markdown()
#         generate_button2 = gr.Button("Generate")
#         gr.Markdown("### Beginner Topics")
#         with gr.Row():
#             for topic in beginner_topics:
#                 button = gr.Button(topic, elem_id="topic-button")
#                 button.click(fn=update_prompt, inputs=gr.Textbox(value=topic, visible=False), outputs=input_box2)
#         gr.Markdown("### Advanced Topics")
#         with gr.Row():
#             for topic in advanced_topics:
#                 button = gr.Button(topic, elem_id="topic-button")
#                 button.click(fn=update_prompt, inputs=gr.Textbox(value=topic, visible=False), outputs=input_box2)
#         generate_button2.click(fn=process_question_no_image, inputs=[input_box2], outputs=output_box2)

# # Combine all interfaces into a tabbed interface
# tabbed_iface = gr.TabbedInterface(
#     interface_list=[iface1, iface2],
#     tab_names=["Ask a Question", "Revise"]
# )

# # Custom CSS styling
# css = """
# #topic-button {
#     background-color: #4CAF50;
#     color: white;
#     border: none;
#     padding: 5px 10px;
#     text-align: center;
#     text-decoration: none;
#     display: inline-block;
#     margin: 2px;
#     cursor: pointer;
#     border-radius: 10px;
#     font-size: 8px;
# }

# #topic-button:hover {
#     background-color: #45a049;
# }
# """

# # Launch the interface
# tabbed_iface.launch(inline=False, inbrowser=True, show_error=True, debug=True)

In [ ]:
# List of topics
beginner_topics = [
    "Variables and Data Types",
    "Control Structures",
    "Functions",
    "Basic Input/Output",
    "Lists and Arrays",
    "Dictionaries",
    "String Manipulation",
    "Basic Error Handling",
    "Basic File Operations",
    "Introduction to Object-Oriented Programming (OOP)",
    "Basic Algorithm Concepts",
    "Introduction to Libraries and Modules"
]

advanced_topics = [
    "Advanced Data Structures (Trees, Graphs)",
    "Algorithms and Complexity (Big O Notation)",
    "Recursion",
    "Object-Oriented Design Principles",
    "Design Patterns",
    "Functional Programming Concepts",
    "Concurrency and Multithreading",
    "Memory Management and Optimization",
    "Network Programming",
    "Database Design and SQL",
    "Web Development Frameworks",
    "Machine Learning Basics"
]

# Create the Gradio interfaces
with gr.Blocks() as iface1:
    with gr.Column():
        gr.Markdown("# Athena - AI Programming Assistant")
        gr.Markdown("## Ask any question related to computer science.")
        image_input = gr.Image(type="pil", label="Upload an Image (Optional)", elem_id="image-input")
        input_box = gr.Textbox(lines=2, placeholder="Type your question here...", label="Question")
        output_box = gr.Markdown()
        generate_button = gr.Button("Generate")
        generate_button.click(fn=process_question_with_image, inputs=[image_input, input_box], outputs=output_box)

with gr.Blocks() as iface2:
    with gr.Column():
        gr.Markdown("Test your skills here by asking Athena to give you revision questions. You can choose your topic from the buttons below or ask about other topics.")
        input_box2 = gr.Textbox(lines=2, placeholder="Generate revision questions on the topic here...", label="Revision Question")
        output_box2 = gr.Markdown()
        generate_button2 = gr.Button("Generate")
        gr.Markdown("### Beginner Topics")
        with gr.Row():
            for topic in beginner_topics:
                button = gr.Button(topic, elem_id="topic-button")
                button.click(fn=update_prompt, inputs=gr.Textbox(value=topic, visible=False), outputs=input_box2)
        gr.Markdown("### Advanced Topics")
        with gr.Row():
            for topic in advanced_topics:
                button = gr.Button(topic, elem_id="topic-button")
                button.click(fn=update_prompt, inputs=gr.Textbox(value=topic, visible=False), outputs=input_box2)
        generate_button2.click(fn=process_question_no_image, inputs=[input_box2], outputs=output_box2)

# Combine all interfaces into a tabbed interface
tabbed_iface = gr.TabbedInterface(
    interface_list=[iface1, iface2],
    tab_names=["Ask a Question", "Revise"],
    theme=gr.Theme.from_hub('HaleyCH/HaleyCH_Theme')
)

# Custom CSS styling
# css = """
# #topic-button {
#     background-color: #4CAF50;
#     color: white;
#     border: none;
#     padding: 5px 10px;
#     text-align: center;
#     text-decoration: none;
#     display: inline-block;
#     margin: 5px;
#     cursor: pointer;
#     border-radius: 10px;
#     font-size: 12px;
#     transition: background-color 0.3s ease;
# }

# #topic-button:hover {
#     background-color: #45a049;
# }

# #image-input {
#     max-width: 100px;
#     margin-bottom: 15px;
# }

# .gr-markdown {
#     font-size: 14px;
# }

# .gr-button {
#     padding: 8px 15px;
#     font-size: 14px;
#     border-radius: 5px;
# }
# """

# Launch the interface
tabbed_iface.launch(inline=False, inbrowser=True, show_error=True, debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://61e6b942f3e6f2fedf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Expanding inputs for image tokens in LLaVa should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1935, in process_api
    result = await s